#Tülomsaş Train Traction Motor

##Ratings

- **Power :** 142-152 kW
- **Voltage :** 800-1100 V
- **Speed :** 374- 3120 rpm
- **Mass:** 1560 kg
- **Motor type:** Three phase IM

[Here](http://www.tulomsas.com.tr/main.php?kid=198&rid=101) you can find the related info about Tülomsaş's motor. 

![motor_picture](http://www.tulomsas.com.tr/upload/resim/101.jpg)

####Breakdown torque is:

\begin{equation}
  T_{bk} = \frac{3p_{1}}{2L_{SC}}(\frac{V_{ph}}{\omega_{1}})^2
   ;L_{SC} = L_{ls}+L_{lr}
\end{equation}

where :
- $p_{1}$ is number of pole-pair,i.e $2p_{1}$ equals to total number of poles 
- $V_{ph}$ is phase voltage
- $L_{SC}$ is total leakage inductance
- $L_{ls}$ and $L_{lr}$ are the stator and rotor leakage inductances,respectively


Since the phase voltages and synchronous speeds are variable, constant V/f technique can be used to keep the max. torque constant during various speeds. Thus, in this design parameters are calculated according to one voltage and frequency which are in between the limit values.

##Stator Core

In [28]:
from math import sqrt,sin,cos,pi
Pn=150e3 #Power
pf= 0.85 #Power-factor cos(phi) from Fig. 16.1
eff = 0.93 #targeted efficiency
n_s=1250 #synchronous speed
Vln=950 #Line-neutral voltage for delta connection
Vph = Vln #phase voltage
m=3 #number of phase
freq=50 #supply frequency
poles=(120*freq)/n_s #number of poles
pp=poles/2 #number of pole pairs
ke= 0.98-0.005*pp #from eq. 15.1
print poles
print ke

4
0.97


**Number of poles** is calculated as $4$

In [29]:
Sgap=ke*Pn/(eff*pf) #Apparent airgap power 
print Sgap
C_0=235e3 # Esson's constant (J/m^3) from fig. 14.14
lmbd=1.4 # Stack aspect ratio from table 15.1
Dis=((2*poles*pp*Sgap)/(pi*lmbd*freq*C_0))**(1/3.0) #stator bore diameter
print Dis

184060.721063
0.384817761458


**Apparent airgap power** is calculated as $184.06 kVA$ 

Therefore **stator bore diameter** is obtained as $0.384  m$

**Slot pitch** is calculated as $0.025 m$

In [30]:
tau=pi*Dis/(2*pp) #pole pitch
L=lmbd*tau #stack lenght
q=4 #slots per pole per phase value(larger q gives better performance-smaller space field harmonics and losses)
tau_s=tau/(m*q) #slot pitch 
print tau_s
print L
print tau

0.025186263591
0.423129228329
0.302235163092
